# Imports

In [56]:
from google.colab import drive
import pandas as pd
import re
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import unittest
import json
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Leitura e combinação dos datasets

## Funções para fazer um tratamento primário dos dados

In [57]:
def remove_emojis(text):
    """
    Remove emojis de um texto.

    Args:
        text (str): O texto a ser processado.

    Returns:
        str: O texto sem emojis.
    """

    if isinstance(text, str):
        emoji_pattern = re.compile(
            "["
            "\U0001F600-\U0001F64F"
            "\U0001F300-\U0001F5FF"
            "\U0001F680-\U0001F6FF"
            "\U0001F1E0-\U0001F1FF"
            "\U00002500-\U00002BEF"
            "\U00002702-\U000027B0"
            "\U00002702-\U000027B0"
            "\U000024C2-\U0001F251"
            "\U0001f926-\U0001f937"
            "\U00010000-\U0010ffff"
            "\u200d"
            "\u2640-\u2642"
            "\u2600-\u2B55"
            "\u23cf"
            "\u23e9"
            "\u231a"
            "\u3030"
            "\ufe0f"
            "]+", flags=re.UNICODE)

        return emoji_pattern.sub(r'', text)
    else:
        return text

def standardize_columns(df):
  """
  Padroniza as colunas de um DataFrame.

  Args:
      df (pd.DataFrame): O DataFrame a ser processado.

  Returns:
      pd.DataFrame: O DataFrame com as colunas padronizadas.
  """
  if 'Resposta\n' in df.columns:
    df = df.rename(columns={'Resposta\n': 'Resposta'})
  if 'Unnamed: 0' in df.columns:
    df = df.rename(columns={'Unnamed: 0': 'No'})
  if 'Comentario do Inacio' in df.columns:
    df = df.drop(columns=['Comentario do Inacio'])

  df = df.dropna(subset=['Pergunta', 'Resposta'], how='all')
  df = df.replace('\n', ' ', regex=True)

  df['Pergunta'] = df['Pergunta'].apply(remove_emojis)
  df['Resposta'] = df['Resposta'].apply(remove_emojis)
  df['No'] = df['No'].fillna(method='ffill')
  df['Intencao'] = df['Intencao'].fillna(method='ffill')

  return df

def organize_no(df, no_start, unique_numbers):
  """
  Organiza os números de conversa a partir do último número do csv anterior

  Args:
      df (pd.DataFrame): O DataFrame a ser processado.
      no_start (int): O número inicial para o No (número do chat).
      unique_numbers (list): A lista de números únicos.

  Returns:
      pd.DataFrame: O DataFrame com os números organizados.
  """
  mapping = {old_value: i + no_start for i, old_value in enumerate(unique_numbers)}
  df['No'] = df['No'].map(mapping)
  return df

## Tratamento primário e concatenação dos datasets

In [ ]:
conv1 = "/content/drive/Shareddrives/grupo3moshi/SPRINT_3/data/Conversas1.csv"
conv2 = "/content/drive/Shareddrives/grupo3moshi/SPRINT_3/data/Conversas2.csv"
dictionary = "/content/drive/Shareddrives/grupo3moshi/SPRINT_3/data/Dicionario.csv"


# Padroniza as colunas para depois conseguir concatenar todos os dataframes
conv1_df = standardize_columns(pd.read_csv(conv1))
conv2_df = standardize_columns(pd.read_csv(conv2))
dictionary_df = pd.read_csv(dictionary)

unique_numbers = conv1_df['No'].unique()
no_start = conv1_df['No'].values[-1:][0]

conv2_df = organize_no(conv2_df, no_start + 1, unique_numbers)

combined_df = pd.concat([conv1_df, conv2_df], ignore_index=True)
combined_df.dropna(inplace = True)

combined_df

# Data augmentation

## Funções de data augmentation

In [59]:
def shuffle_words(sentence):
    """
    Embaralha as palavras em uma frase

    Args:
      sentence (str): A frase a ser embaralhada.

    Returns:
      str: A frase embaralhada.
    """
    words = sentence.split()
    random.shuffle(words)
    return ' '.join(words)

def get_synonyms(word):
    """
    Retorna os sinônimos de uma palavra

    Args:
      word (str): A palavra a ser pesquisada os sinônimos dela.

    Returns:
      list: A lista de sinônimos.
    """
    synonyms = set()
    for syn in wordnet.synsets(word, lang='por'):
        for lemma in syn.lemmas(lang='por'):
            synonyms.add(lemma.name().replace("_", " "))
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

def synonym_replacement(sentence, n=2):
    """
    Substitui aleatoriamente n palavras por seus sinônimos

    Args:
      sentence (str): A frase a ter palavras substituídas por sinônimos.
      n (int): O número de palavras a serem substituídas.

    Returns:
      str: A frase com sinônimos substituídos.
    """
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if len(get_synonyms(word)) > 0]))
    random.shuffle(random_word_list)

    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    return ' '.join(new_words)

def random_deletion(words, p=0.14):
    """
    Remove com a probabilidade p as palavras de uma frase

    Args:
      words (str): A frase a ter palavras removidas.
      p (float): A probabilidade de remoção de palavras.

    Returns:
      str: A frase com palavras removidas.
    """
    words = words.split()
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)
    return ' '.join(new_words)

## Pipeline de data augmentation

In [ ]:
def augment_data(df):
  """
  Aplica as técnicas de data augmentation ao DataFrame.

  Args:
      df (pd.DataFrame): O DataFrame a ser processado.

  Returns:
      pd.DataFrame: O DataFrame com data augmentation aplicada.
  """

  unique_numbers = df['No'].unique()
  no_start = conv2_df['No'].values[-1:][0]

  df_shuffled = pd.DataFrame()
  df_shuffled = df.copy()
  # Aplicando o embaralhamento de palavras
  df_shuffled['Pergunta'] = df['Pergunta'].apply(lambda x: shuffle_words(x))
  df_shuffled['Resposta'] = df['Resposta'].apply(lambda x: shuffle_words(x))
  df_shuffled = organize_no(df_shuffled, no_start + 1, unique_numbers)

  df_synonym = pd.DataFrame()
  df_synonym = df.copy()
  # Aplicando a substituição de sinônimos
  df_synonym['Pergunta'] = df['Pergunta'].apply(lambda x: synonym_replacement(x))
  df_synonym['Resposta'] = df['Resposta'].apply(lambda x: synonym_replacement(x))
  df_synonym = organize_no(df_synonym, no_start * 2 + 1, unique_numbers)

  df_deletion = pd.DataFrame()
  df_deletion = df.copy()
  # Aplicando a deleção aleatória
  df_deletion['Pergunta'] = df['Pergunta'].apply(lambda x: random_deletion(x))
  df_deletion['Resposta'] = df['Resposta'].apply(lambda x: random_deletion(x))
  df_deletion = organize_no(df_deletion, no_start * 3 + 1, unique_numbers)

  augmented_df = pd.concat([df, df_shuffled, df_synonym, df_deletion], ignore_index=True)
  return augmented_df

augmented_df = augment_data(combined_df)
augmented_df

# Substituição de palavras japonesas

In [61]:
substituicoes = dict(zip(dictionary_df['Palavra '], dictionary_df['Significado']))

def substituir_palavras(texto, substituicoes):
    """
    Substitui palavras em um texto com base no dicionário mandado pelo Inácio

    Args:
      texto (str): O texto a ser processado.
      substituicoes (dict): O dicionário de substituições.

    Returns:
      str: O texto com palavras substituídas.
    """
    for palavra, significado in substituicoes.items():
        texto = texto.replace(palavra, significado)
    return texto

augmented_df['Pergunta'] = augmented_df['Pergunta'].apply(lambda x: substituir_palavras(x, substituicoes))

# Preparação da tabela para treinamento do modelo

In [ ]:
def combine_questions_and_answers(df):
  """
  Combina as respostas e perguntas em uma única coluna, de uma forma que tenha
  o histórico do chat até aquele momento.

  Args:
      df (pd.DataFrame): O DataFrame a ser processado.

  Returns:
      pd.DataFrame: O DataFrame com as perguntas e respostas combinadas.
  """
  unique_numbers = df['No'].unique()

  # Para cada chat, junta as perguntas e respostas, criando um histórico da
  # conversa até aquela pergunta
  for number in unique_numbers:
    full_string = ""
    first_line = True
    rows = df.loc[df['No'] == number, ['Pergunta', 'Resposta']]
    for index, row in rows.iterrows():
      if first_line:
        full_string += f"{row['Pergunta']} {row['Resposta']} "
        first_line = False
        continue
      if pd.notna(row['Pergunta']):
        full_string += f"{row['Pergunta']} "
      df.loc[index, 'Pergunta'] = full_string
      if pd.notna(row['Resposta']):
        full_string += f"{row['Resposta']} "

  return df

return_df = combine_questions_and_answers(augmented_df)

return_df

# Testes

## Teste funções de tratamento primário

In [63]:
class TestPrimaryTreatmentFunctions(unittest.TestCase):

    def test_remove_emojis(self):
        self.assertEqual(remove_emojis("Olá 😃!"), "Olá !")
        self.assertEqual(remove_emojis("🚀 Vamos para Marte!"), " Vamos para Marte!")
        self.assertEqual(remove_emojis("Sem emojis aqui."), "Sem emojis aqui.")

        self.assertEqual(remove_emojis(""), "")

        self.assertEqual(remove_emojis("Texto sem emojis"), "Texto sem emojis")

        self.assertEqual(remove_emojis("12345 @$%*!"), "12345 @$%*!")

        self.assertEqual(remove_emojis(12345), 12345)

    def test_standardize_columns(self):
        df = pd.DataFrame({
            'Unnamed: 0': [1, 2, 3],
            'Pergunta': ['Qual o seu nome?', None, 'De onde você é?'],
            'Resposta\n': ['Meu nome é Pedro', 'Eu sou do Brasil', None],
            'Comentario do Inacio': ['irrelevante', 'irrelevante', 'irrelevante'],
            'Intencao': ['Saudacao', None, 'Origem']
        })

        df_padronizado = standardize_columns(df)

        self.assertIn('No', df_padronizado.columns)
        self.assertIn('Resposta', df_padronizado.columns)
        self.assertNotIn('Comentario do Inacio', df_padronizado.columns)

        self.assertEqual(df_padronizado['Intencao'].iloc[1], 'Saudacao')

        self.assertNotIn('🚀', df_padronizado['Pergunta'].iloc[0])
        self.assertNotIn('😃', df_padronizado['Resposta'].iloc[0])

    def test_organize_no(self):
        df = pd.DataFrame({
            'No': [10, 10, 20, 20],
            'Pergunta': ['Como está?', 'Qual o seu nome?', 'De onde você é?', 'Qual sua idade?'],
            'Resposta': ['Bem', 'Pedro', 'Brasil', '20 anos']
        })

        unique_numbers = [10, 20]
        df_organizado = organize_no(df, no_start=100, unique_numbers=unique_numbers)

        self.assertEqual(df_organizado['No'].iloc[0], 100)
        self.assertEqual(df_organizado['No'].iloc[2], 101)

## Teste funções de data augmentation

In [64]:
class TestTextAugmentation(unittest.TestCase):

    def test_shuffle_words(self):
        sentence = "Isto é um teste"
        shuffled = shuffle_words(sentence)
        self.assertNotEqual(shuffled, sentence)
        self.assertCountEqual(shuffled.split(), sentence.split())

        single_word = "teste"
        self.assertEqual(shuffle_words(single_word), single_word)

        empty_sentence = ""
        self.assertEqual(shuffle_words(empty_sentence), "")

    def test_get_synonyms(self):
        synonyms = get_synonyms("feliz")
        self.assertTrue(isinstance(synonyms, list))
        self.assertIn("contente", synonyms)

        synonyms_no_result = get_synonyms("inexistente")
        self.assertEqual(synonyms_no_result, [])

        synonyms_common_word = get_synonyms("bom")
        self.assertTrue(len(synonyms_common_word) > 0)

    def test_synonym_replacement(self):
        random.seed(42)

        sentence = "Este é um teste simples"
        replaced_sentence = synonym_replacement(sentence, n=1)

        self.assertNotEqual(replaced_sentence, sentence)
        self.assertEqual(len(replaced_sentence.split()), len(sentence.split()))

    def test_random_deletion(self):
        random.seed(42)

        sentence = "Este é um teste de remoção"
        deleted_sentence = random_deletion(sentence, p=0.5)

        self.assertNotEqual(deleted_sentence, sentence)
        self.assertLess(len(deleted_sentence.split()), len(sentence.split()))

        no_deletion_sentence = random_deletion(sentence, p=0)
        self.assertEqual(no_deletion_sentence, sentence)

        full_deletion_sentence = random_deletion(sentence, p=1)
        self.assertEqual(full_deletion_sentence, "")

## Teste pipeline de data augmentation

In [65]:
class TestAugmentData(unittest.TestCase):

    def setUp(self):
        self.df = pd.DataFrame({
            'No': [1, 2],
            'Pergunta': ['Como você está?', 'Qual é o seu nome?'],
            'Resposta': ['Estou bem, obrigado!', 'Meu nome é ChatGPT.'],
            'Intencao': ['saudacao', 'identificacao']
        })

    def test_augment_data(self):

        augmented_df = augment_data(self.df)

        self.assertEqual(len(augmented_df), len(self.df) * 4)

        self.assertTrue(all(col in augmented_df.columns for col in ['No', 'Pergunta', 'Resposta', 'Intencao']))

## Teste substituição de palavras japonesas

In [66]:
class TestDictionary(unittest.TestCase):
  def test_japanese_word_replacement(self):
        sentence = "Oi kakunin tudo gomen?"
        replaced_sentence = substituir_palavras(sentence, substituicoes)

        self.assertNotEqual(replaced_sentence, sentence)
        self.assertEqual(replaced_sentence, "Oi verificacao tudo desculpe?")

## Teste concatenação perguntas e respostas

In [74]:
class TestCombineQuestionsAndAnswers(unittest.TestCase):

    def setUp(self):
        self.df = pd.DataFrame({
            'No': [1, 1, 1, 2, 2],
            'Pergunta': ['Boa tarde Acabei de fazer a transferência de 22+23, Total de 45Yenes',
                         'Poderia fazer a remessa de 22yenes para o BB RR obrigado E de 23yenes para o AAA MMM caixa econômica federal Obrigado',
                         "Obrigado Uma boa noite Deus abençoe", 'Boa noite, fiz uma transferência de 30.000 yenes, é pra LLL NNN . Arigatou', 'Me confirma quanto deu em real? \"Bom dia! Me confirma o valor dessa transferência de ontem por favor \"'],
            'Resposta': ['Boa tarde', "Iremos processar a sua solicitacao. Muito obrigada e otima tarde.", "Qualquer dúvida estamos à disposição. Obrigado.",
                         "Obrigado pela confirmação! Vamos processar a sua remessa.", "Bom dia. Ir chegar o valor de BRL 960.96 reais. \
                         Perdao, pela nova cotacao, chegara o valor de BRL 965.81. Qualquer dúvida estamos à disposição. Obrigado. Um otimo dia"],
            'Intencao': ['Pedido de envio via metodo "ByPhone"', 'Pedido de envio via metodo "ByPhone"', 'Pedido de envio via metodo "ByPhone"', 'Pedido de envio via metodo "ByPhone"',
                         'Confirmacao de cambio/taxas']
        })

    def test_combine_questions_and_answers(self):
        combined_df = combine_questions_and_answers(self.df.copy())

        expected_history_chat_1 = [
            "Boa tarde Acabei de fazer a transferência de 22+23, Total de 45Yenes",
            "Boa tarde Acabei de fazer a transferência de 22+23, Total de 45Yenes Boa tarde Poderia fazer a remessa de 22yenes para o BB RR obrigado E de 23yenes para o AAA MMM caixa econômica federal Obrigado ",
            "Boa tarde Acabei de fazer a transferência de 22+23, Total de 45Yenes Boa tarde Poderia fazer a remessa de 22yenes para o BB RR obrigado E de 23yenes para o AAA MMM caixa econômica federal Obrigado Iremos processar a sua solicitacao. Muito obrigada e otima tarde. Obrigado Uma boa noite Deus abençoe "
        ]

        for i, expected in enumerate(expected_history_chat_1):
            self.assertEqual(combined_df.loc[i, 'Pergunta'], expected)

        expected_history_chat_2 = [
            "Boa noite, fiz uma transferência de 30.000 yenes, é pra LLL NNN . Arigatou",
            "Boa noite, fiz uma transferência de 30.000 yenes, é pra LLL NNN . Arigatou Obrigado pela confirmação! Vamos processar a sua remessa. Me confirma quanto deu em real? \"Bom dia! Me confirma o valor dessa transferência de ontem por favor \" "
        ]

        for i, expected in enumerate(expected_history_chat_2, start=3):
            self.assertEqual(combined_df.loc[i, 'Pergunta'], expected)

## Execução dos testes

In [75]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....<ipython-input-57-1231c6b11aad>:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['No'] = df['No'].fillna(method='ffill')
<ipython-input-57-1231c6b11aad>:62: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Intencao'] = df['Intencao'].fillna(method='ffill')
.....
----------------------------------------------------------------------
Ran 10 tests in 0.049s

OK


# Exportar csv para o drive

In [69]:
return_df.to_csv('/content/drive/Shareddrives/grupo3moshi/SPRINT_4/augmented_data_sprint_4.csv', index=False)